In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the dataset
df = pd.read_csv('/content/Dataset/Illinois_10_years_data.csv')

# Drop rows with missing values
df.dropna(inplace=True)

# Downsample the data to reduce size (e.g., sample 1% of the data)
df = df.sample(frac=0.01, random_state=42).reset_index(drop=True)

# Convert integer features to int32 for memory efficiency
int_columns = ['Year', 'Quarter', 'Month', 'Day_of_Month', 'Day_of_Week', 'Scheduled_Departure_Time', 'Scheduled_Departure_Time_Minutes', 'Target']
for col in int_columns:
    df[col] = df[col].astype(np.int32)

# Convert continuous numeric features to float32
float_columns = ['Taxi_Out_Time_Minutes', 'Flight_Distance_Miles', 'Air_Temperature_Fahrenheit', 'Dew_Point_Temperature_Fahrenheit',
                 'Relative_Humidity_Percent', 'Wind_Direction_Degrees', 'Wind_Speed_Knots', 'Hourly_Precipitation_Inches',
                 'Pressure_Altimeter_Inches', 'Sea_Level_Pressure_Millibar', 'Visibility_Miles', 'Sky_Level_1_Altitude_Feet',
                 'Apparent_Temperature_Fahrenheit']
for col in float_columns:
    df[col] = df[col].astype(np.float32)

# Drop unnecessary columns
df = df.drop(['Origin_State_Name', 'Departure_Datetime', 'Departure_Delay_Minutes'], axis=1)

# Define categorical columns
categorical_columns = ['Operating_Carrier_Code', 'Tail_Number', 'Origin_Airport_ID', 'Origin_Airport_Code', 'Destination_Airport_Code', 'Destination_State_Name', 'Sky_Cover_Level_1']

# Separate high-cardinality and low-cardinality categorical columns
high_cardinality_columns = ['Tail_Number', 'Origin_Airport_Code', 'Destination_Airport_Code']
low_cardinality_columns = [col for col in categorical_columns if col not in high_cardinality_columns]

# Apply label encoding to high-cardinality categorical features
label_encoders = {}
for col in high_cardinality_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Apply one-hot encoding to low-cardinality categorical features
df = pd.get_dummies(df, columns=low_cardinality_columns, drop_first=True)

# Define the years for each dataset
train_years = [2014, 2015, 2016, 2017, 2018, 2019]
val_years = [2020, 2021, 2022]
test_years = [2023, 2024]

# Create train, validation, and test sets
train_df = df[df['Year'].isin(train_years)].reset_index(drop=True)
val_df = df[df['Year'].isin(val_years)].reset_index(drop=True)
test_df = df[df['Year'].isin(test_years)].reset_index(drop=True)

# Separate features and target variable
X_train = train_df.drop('Target', axis=1)
y_train = train_df['Target']
X_val = val_df.drop('Target', axis=1)
y_val = val_df['Target']
X_test = test_df.drop('Target', axis=1)
y_test = test_df['Target']

# Standardize numeric features to improve SVM model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Train a simple SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Evaluate on validation set
y_val_pred = svm_model.predict(X_val_scaled)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report on Validation Set:\n", classification_report(y_val, y_val_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_val_pred))

# Evaluate on test set
y_test_pred = svm_model.predict(X_test_scaled)
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nClassification Report on Test Set:\n", classification_report(y_test, y_test_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))



ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

**For the Validation Set:**

* We get an accuracy of 86.3% which seems high, but it's misleading. This accuracy reflects that the model is heavily biased towards predicting the majority class (no delay), which dominates the dataset.

* Class 0 (No Delay): The model has high precision (0.86) and recall (1.00), meaning it correctly identifies non-delayed flights.

* Class 1 (Delay): Precision and recall are both 0, indicating that the model fails to identify any delays. This likely occurs because of class imbalance—there are far fewer delayed flights compared to non-delayed ones, so the model defaults to predicting "No Delay" for all instances.

* The confusion matrix shows that all 946 delayed flights were classified as non-delayed, with no true positives for the delay class.

**For the test set:**

* : We get an accuracy of 77%, which again seems reasonable at first glance but is similarly biased.


Thus, we need to Oversample the minority class (delays) or undersample the majority class (no delay). We can use the technique SMOTE (Synthetic Minority Over-sampling Technique) to generate synthetic samples for the minority class.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('/content/Dataset/Illinois_10_years_data.csv')

# Drop rows with missing values
df.dropna(inplace=True)

# Downsample to 10% for performance (adjust as needed)
df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)

# Convert integer features to int32 for memory efficiency
int_columns = ['Year', 'Quarter', 'Month', 'Day_of_Month', 'Day_of_Week', 'Scheduled_Departure_Time', 'Scheduled_Departure_Time_Minutes', 'Target']
for col in int_columns:
    df[col] = df[col].astype(np.int32)

# Convert continuous numeric features to float32
float_columns = ['Taxi_Out_Time_Minutes', 'Flight_Distance_Miles', 'Air_Temperature_Fahrenheit', 'Dew_Point_Temperature_Fahrenheit',
                 'Relative_Humidity_Percent', 'Wind_Direction_Degrees', 'Wind_Speed_Knots', 'Hourly_Precipitation_Inches',
                 'Pressure_Altimeter_Inches', 'Sea_Level_Pressure_Millibar', 'Visibility_Miles', 'Sky_Level_1_Altitude_Feet',
                 'Apparent_Temperature_Fahrenheit']
for col in float_columns:
    df[col] = df[col].astype(np.float32)

# Drop unnecessary columns
df = df.drop(['Origin_State_Name', 'Departure_Datetime', 'Departure_Delay_Minutes'], axis=1)

# Define categorical columns
categorical_columns = ['Operating_Carrier_Code', 'Tail_Number', 'Origin_Airport_ID', 'Origin_Airport_Code', 'Destination_Airport_Code', 'Destination_State_Name', 'Sky_Cover_Level_1']

# Apply label encoding to high-cardinality categorical features
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Separate features and target
X = df.drop('Target', axis=1)
y = df['Target']

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the SVM model
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test_scaled)
y_pred_prob = svm_model.predict_proba(X_test_scaled)[:,1]

# Classification Report and Confusion Matrix
print("Classification Report:\n", classification_report(y_test, y_pred))
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", conf_matrix)

# Visualization Metrics
# 1. Confusion Matrix Heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['No Delay', 'Delay'], yticklabels=['No Delay', 'Delay'])
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# 2. ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='blue', label='AUC = %0.2f' % roc_auc)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.show()

# 3. Precision-Recall Curve
from sklearn.metrics import precision_recall_curve
precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)

plt.figure(figsize=(6, 4))
plt.plot(recall, precision, color='purple')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.show()
